# Gemini
No fine-tuning, but taking advantage of the large context window instead.
* https://ai.google.dev/tutorials/tuning_quickstart_python
* https://github.com/google/generative-ai-python/tree/v0.3.0/google/generativeai
* https://ai.google.dev/gemini-api/docs/get-started/python
* `gemini-pro`: optimized for text-only prompts.
* `gemini-pro-vision`: optimized for text-and-images prompts.

In [1]:
with open('gemini_api_key.txt','r') as f:
    my_gemini_api_key = f.read()

In [30]:
import os
import glob
import pandas as pd

datasets = sorted(glob.glob(os.path.join(os.getcwd(), 
                                            'data', 
                                            'DRS', 
                                            'dataset', 
                                            'all_drs', 
                                            'dataset_gemini raw data_*.parquet')), 
				key=os.path.getctime, 
				reverse=True)
df_dataset = pd.read_parquet(datasets[0])
df_dataset = df_dataset.reset_index()

In [3]:
from pathlib import Path
import hashlib
import google.generativeai as genai

genai.configure(api_key=my_gemini_api_key)

# Set up the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 8192,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]

model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              safety_settings=safety_settings)

/home/victor/anaconda3/envs/google_gemini/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [34]:
df_dataset.iloc[100]["0"]

"title:\n Public Aircraft Operations Manned and Unmanned\n\ndocument context and subtitles:\n ; '13 AC FEEDBACK FORM. For your convenience; the AC Feedback Form is the last page of this AC. Note any deficiencies found; clarifications needed; or suggested improvements regarding the contents of this AC on the Feedback Form.'; 'Appendix B.   CONTACT INFORMATION'; 'B.4 UAS QUESTIONS. If you have a UAS question; please contact the Unmanned Aircraft Systems Safety and Integration Division (AUS-400) at https://www.faa.gov/uas/; or at the address below:\n\ncontent:\n  If you have a UAS question please contact th"

In [35]:
convo = model.start_chat(history=[
  {
    "role": "user",
    "parts": [df_dataset.iloc[100]["0"]]
  },
  {
    "role": "user",
    "parts": [df_dataset.iloc[101]["0"]]
  },
])

In [36]:
convo.send_message("What is the context of those documents?")

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=glm.GenerateContentResponse({'candidates': [{'content': {'parts': [{'text': '## Context of the Documents\n\nBased on the provided titles and subtitles, the context appears to be related to **regulations and guidance for operating both manned and unmanned public aircraft**. This likely includes information on:\n\n* **Public aircraft operations:**  This would cover the rules and procedures for operating aircraft owned and operated by government entities for public purposes. \n* **Unmanned Aircraft Systems (UAS):** This section would specifically focus on the regulations and safety procedures for operating drones and other unmanned aircraft. \n* **Contact information:** The document provides resources for further questions or clarification, specifically directing UAS inquiries to the Unmanned Aircraft Systems Safety and Integration Division (AUS-400).\n\n**Additional Observations:**\n\n* The presence of a feed

In [37]:
print(convo.last.text)

AttributeError: 'NoneType' object has no attribute 'text'

## GenerativeModel

### Import packages

In [ ]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

## Generate text from text inputs

In [ ]:
response = model.generate_content("What is the meaning of life?")
to_markdown(response.text)
response.prompt_feedback
response.candidates